In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('UWRL_river_velocimetry_dataset.csv')

In [ ]:
df['v_pos'] =  [0 if pd.isna(x) else (-1 if x < 0 else 1) for x in df['v_x']]
df.datetime = pd.to_datetime(df.datetime)
print(df.columns)
df

In [ ]:
df['year'] = [int(x.year) for x in df.datetime]
df['month'] = [int(x.month) for x in df.datetime]
df['day'] = [int(x.day) for x in df.datetime]
df['hour'] = [int(x.hour) for x in df.datetime]

In [ ]:
sunrise, sunset = [], []
for month in sorted(df.month.unique()):
    df_month = df[df.month == month]

    for day in sorted(df_month.day.unique()):
        df_day = df_month[df_month.day == day]

        times_sorted = sorted(df_day.datetime.unique())

        sunrise.append(times_sorted.pop(0))
        try:
            sunset.append(times_sorted.pop(-1))
        except:
            continue

df['sunrise_sunset'] = ['sunrise' if x in sunrise else 'sunset' if x in sunset else 'no' for x in df.datetime]

In [ ]:
df_ss = df.groupby(['sunrise_sunset', 'spacial_location', 'v_pos']).size().reset_index(name='counts')

for x in df_ss.sunrise_sunset.unique():

    print(x)

    sns.barplot(
        data=df_ss[df_ss.sunrise_sunset == x],
        x='spacial_location',
        y='counts',
        hue='v_pos'
    )
    plt.legend(labels=['downstream', 'NaN', 'upstream'], bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    plt.close()


In [ ]:
df_cond = df.groupby(['conditions', 'spacial_location', 'v_pos']).size().reset_index(name='counts')

for x in df_cond.conditions.unique():

    print(x)

    sns.barplot(
        data=df_cond[df_cond.conditions == x],
        x='spacial_location',
        y='counts',
        hue='v_pos'
    )
    plt.legend(labels=['downstream', 'NaN', 'upstream'], bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
    plt.close()
